<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/Claude_3_5_Sonnet_to_gpt_4o_mini_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Claude 3.5 Sonnet to gpt-4o-mini - part of the `gpt-prompt-engineer` repo

This notebook gives you the ability to go from Claude 3.5 Sonnet to GPT-4o-mini -- reducing costs massively while keeping quality high.

By Matt Shumer (https://twitter.com/mattshumer_)

Github repo: https://github.com/mshumer/gpt-prompt-engineer

In [ ]:
!pip install openai

OPENAI_API_KEY = "YOUR API KEY HERE" # enter your OpenAI API key here
ANTHROPIC_API_KEY = "YOUR API KEY HERE" # enter your Anthropic API key here

In [ ]:
import re
import json
import requests
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

def generate_candidate_prompts(task, prompt_example, response_example):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-5-sonnet-20240620',
        "max_tokens": 4000,
        "temperature": .5,
        "system": """<task>Given an example training sample, create seven additional samples for the same task that are even better. Each example should contain a <prompt> and a <response>.</task>

<rules>
1. Ensure the new examples are diverse and unique from one another.
2. They should all be perfect. If you make a mistake, this system won't work.
</rules>

Respond in this format:
<response_format>
<example_one>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_one>

<example_two>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_two>

...
</response_format>""",
        "messages": [
            {"role": "user", "content": f"""<training_task>{task}</training_task>

<prompt_example>
{prompt_example}
</prompt_example>

<response_example>
{response_example}
</response_example>"""},
        ]
    }


    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

    response_text = response.json()['content'][0]['text']

    # Parse out the prompts and responses
    prompts_and_responses = []
    examples = re.findall(r'<example_\w+>(.*?)</example_\w+>', response_text, re.DOTALL)
    for example in examples:
        prompt = re.findall(r'<prompt>(.*?)</prompt>', example, re.DOTALL)[0].strip()
        response = re.findall(r'<response>(.*?)</response>', example, re.DOTALL)[0].strip()
        prompts_and_responses.append({'prompt': prompt, 'response': response})

    return prompts_and_responses

def generate_system_prompt(task, prompt_examples):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-5-sonnet-20240620',
        "max_tokens": 1000,
        "temperature": .5,
        "system": """<your_role>Given a user-description of their <task> a set of prompt / response pairs (it'll be in JSON for easy reading) for the types of outputs we want to generate given inputs, write a fantastic system prompt that describes the task to be done perfectly.</your_role>

<rules>
1. Do this perfectly.
2. Respond only with the system prompt, and nothing else. No other text will be allowed.
</rules>

Respond in this format:
<system_prompt>
WRITE_SYSTEM_PROMPT_HERE
</system_prompt>""",
        "messages": [
            {"role": "user", "content": f"""<task>{task}</task>

<prompt_response_examples>
{str(prompt_examples)}
</prompt_response_examples>"""},
        ]
    }


    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

    response_text = response.json()['content'][0]['text']

    # Parse out the prompt
    system_prompt = response_text.split('<system_prompt>')[1].split('</system_prompt>')[0].strip()

    return system_prompt

def test_mini(generated_examples, prompt_example, system_prompt):
    messages = [{"role": "system", "content": system_prompt}]

    for example in generated_examples:
        messages.append({"role": "user", "content": example['prompt']})
        messages.append({"role": "assistant", "content": example['response']})

    messages.append({"role": "user", "content": prompt_example.strip()})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=2000,
        temperature=0.5
    )

    response_text = response.choices[0].message.content

    return response_text

def run_mini_conversion_process(task, prompt_example, response_example):
    print('Generating the prompts / responses...')
    # Generate candidate prompts
    generated_examples = generate_candidate_prompts(task, prompt_example, response_example)

    print('Prompts / responses generated. Now generating system prompt...')

    # Generate the system prompt
    system_prompt = generate_system_prompt(task, generated_examples)

    print('System prompt generated:', system_prompt)

    print('\n\nTesting the new prompt on GPT-4o-mini, using your input example...')
    # Test the generated examples and system prompt with the GPT-4o-mini model
    mini_response = test_mini(generated_examples, prompt_example, system_prompt)

    print('GPT-4o-mini responded with:')
    print(mini_response)

    print('\n\n!! CHECK THE FILE DIRECTORY, THE PROMPT IS NOW SAVED THERE !!')

    # Create a dictionary with all the relevant information
    result = {
        "task": task,
        "initial_prompt_example": prompt_example,
        "initial_response_example": response_example,
        "generated_examples": generated_examples,
        "system_prompt": system_prompt,
        "mini_response": mini_response
    }

    # Save the GPT-4o-mini prompt to a Python file
    with open("gpt4o_mini_prompt.py", "w") as file:
        file.write('system_prompt = """' + system_prompt + '"""\n\n')

        file.write('messages = [\n')
        for example in generated_examples:
            file.write('    {"role": "user", "content": """' + example['prompt'] + '"""},\n')
            file.write('    {"role": "assistant", "content": """' + example['response'] + '"""},\n')

        file.write('    {"role": "user", "content": """' + prompt_example.strip() + '"""}\n')
        file.write(']\n')

    return result

## Fill in your task, prompt_example, and response_example here. Make sure you keep the quality really high here... this is the most important step!

In [ ]:
task = "refactoring complex code"

prompt_example = """def calculate_total(prices, tax, discount, shipping_fee, gift_wrap_fee, membership_discount):

    total = 0

    for i in range(len(prices)):

        total += prices[i]

    if membership_discount != 0:

        total = total - (total * (membership_discount / 100))

    if discount != 0:

        total = total - (total * (discount / 100))

    total = total + (total * (tax / 100))

    if total < 50:

        total += shipping_fee

    else:

        total += shipping_fee / 2

    if gift_wrap_fee != 0:

        total += gift_wrap_fee * len(prices)

    if total > 1000:

        total -= 50

    elif total > 500:

        total -= 25

    total = round(total, 2)

    if total < 0:

        total = 0

    return total"""

response_example = """def calculate_total(prices, tax_rate, discount_rate, shipping_fee, gift_wrap_fee, membership_discount_rate):

    def apply_percentage_discount(amount, percentage):

        return amount * (1 - percentage / 100)

    def calculate_shipping_fee(total):

        return shipping_fee if total < 50 else shipping_fee / 2

    def apply_tier_discount(total):

        if total > 1000:

            return total - 50

        elif total > 500:

            return total - 25

        return total

    subtotal = sum(prices)

    subtotal = apply_percentage_discount(subtotal, membership_discount_rate)

    subtotal = apply_percentage_discount(subtotal, discount_rate)



    total = subtotal * (1 + tax_rate / 100)

    total += calculate_shipping_fee(total)

    total += gift_wrap_fee * len(prices)



    total = apply_tier_discount(total)

    total = max(0, round(total, 2))



    return total"""

### Now, let's run this system and get our new prompt! At the end, you'll see a new file pop up in the directory that contains everything you'll need to reduce your costs while keeping quality high w/ gpt-4o-mini!

In [ ]:
result = run_mini_conversion_process(task, prompt_example, response_example)